In [1]:
import torch
from transformers import AutoTokenizer, OPTForCausalLM
import pandas as pd
import numpy as np
from typing import Dict, List
import json
from pathlib import Path
import logging
from tqdm import tqdm
import argparse,os
import csv
from diffusers import UNet2DModel, AutoPipelineForText2Image, UNet2DModel
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler
from PIL import Image
from diffusers import LMSDiscreteScheduler
from tqdm.auto import tqdm
from torch import autocast

/home/hxxzhang/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hxxzhang/miniconda3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/hxxzhang/miniconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/hxxzhang/miniconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pyt

# Tokenize safe phrases from safe prompts

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag


def tokenize_and_clean(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove symbols and punctuation using regex
    tokens = [re.sub(r'\W+', '', token) for token in tokens if re.sub(r'\W+', '', token)]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Get parts of speech tags
    pos_tags = pos_tag(tokens)
    
    # Remove adpositions (prepositions and postpositions)
    tokens = {word for word, pos in pos_tags if pos not in ['IN']}
    
    return tokens
def chunk_phrases(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # POS tagging
    pos_tags = nltk.pos_tag(tokens)
    
    # Define a chunk grammar
    # Here we define chunk patterns for noun phrases (NP)
    grammar = r"""
      NP: {<DT>?<JJ>*<NN.*>}  # Chunk sequences of DT, JJ, NN
          {<NN.*><NN.*>}      # Chunk consecutive nouns
    """
    
    # Create a chunk parser
    chunk_parser = nltk.RegexpParser(grammar)
    
    # Parse the POS tagged tokens to chunk phrases
    tree = chunk_parser.parse(pos_tags)
    
    # Extract and return the chunked phrases
    chunked_phrases = set()
    for subtree in tree:
        if type(subtree) == nltk.Tree:
            phrase = " ".join([token for token, pos in subtree.leaves()])
            chunked_phrases.add(phrase)
    
    return chunked_phrases

In [9]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/hxxzhang/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
# Phrases from safe prompts can be considered safe
sfw_prompts=pd.read_csv("PATH to NORMAL PROMPT")['prompt'].tolist()
sfw_phrase=set()
for prompt in sfw_prompts:
    sfw_phrase  = sfw_phrase.union(chunk_phrases(prompt))

with open("PATH to SAVE SAFE PHRASES",'w') as file:
    for sfw in sfw_phrase:
        file.write(sfw+'\n')

In [ ]:
max_len = 0
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
torch_device = "cuda"
text_encoder = text_encoder.to(torch_device)

for i in sfw_phrase:
    tokenized_sfw = tokenizer(i,truncation=True,return_tensors="pt")
    encode_sfw = text_encoder(tokenized_sfw.input_ids.to(torch_device))
    max_len=max(max_len,encode_sfw.last_hidden_state.shape[1])
print("MAX LEN of text embedding is:",max_len)

# Generate SFW phrases with NSFW phrases

In [ ]:
# Utilize gpt-4o to generate SFW phrases with given NSFW phrases
from openai import OpenAI
OPENAI_API_KEY = "OPENAI API KEY"
client = OpenAI(api_key=OPENAI_API_KEY)
ans=[]
for prompt in nsfw:
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a NSFW and SFW safety assistant. Return the safe and benign opposite meaning phrase of the given phrase. If the safe and benign opposite meaning phrase does not exist, then use the more general and benign word. Only return the phrase no other words, no other response."},
        {"role": "user", "content": prompt}
    ]
    )
    ans.append(completion.choices[0].message)

ChatCompletionMessage(content='No bad content detected.', refusal=None, role='assistant', function_call=None, tool_calls=None)
